In [74]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd
import re
from tqdm import tqdm

In [75]:
partA = 'https://bugzilla.mozilla.org/buglist.cgi?bug_status=RESOLVED&bug_status=VERIFIED&bug_status=CLOSED&bug_type=defect&chfield=anything&chfieldfrom=2018-10-23&classification=Client%20Software&classification=Developer%20Infrastructure&classification=Components&classification=Server%20Software&classification=Other&list_id=15773789&query_format=advanced&resolution=FIXED&short_desc='

partB = '&short_desc_type=allwordssubstr&order=bug_status%2Cpriority%2Cassigned_to%2Cbug_id&limit=0'

# keyword = ['XSS' , 'overflow','crash'] 

keyword = ['xss',
 'overflow',
 'oom',
 'out of bound',
 'valid',
 'memori',
 'memory',
 'inject',
 'use after fre',
 'csrf',
 'travers',
 'pointer',
 'authent',
 'author',
 'danger',
 'permiss',
 'critic',
 'credenti',
 'xml',
 'xml external ent',
 'xxe',
 'deseri',
 'privileg',
 'secur',
 'vulner',
 'thread',
 'crash']



In [76]:
# Adding '%20' in place of space in the keywords

checkedKeywordsList = []
for w in keyword:
    if ' ' in w:
        temp = w.split(' ')
        word = ''
        for t in temp:
            word = word + t + '%20'
        checkedKeywordsList.append(word)
#         print(word)
    else:
        checkedKeywordsList.append(w)

checkedKeywordsList

['xss',
 'overflow',
 'oom',
 'out%20of%20bound%20',
 'valid',
 'memori',
 'memory',
 'inject',
 'use%20after%20fre%20',
 'csrf',
 'travers',
 'pointer',
 'authent',
 'author',
 'danger',
 'permiss',
 'critic',
 'credenti',
 'xml',
 'xml%20external%20ent%20',
 'xxe',
 'deseri',
 'privileg',
 'secur',
 'vulner',
 'thread',
 'crash']

In [77]:
# creating a blank dataframe
col = ['keyword','bugId','summary','product','component']
taskDf = pd.DataFrame(columns = {'keyword','bugId','summary','product','component'}, index = None)
wordWiseResult = pd.DataFrame(columns = {'word', 'Results found'}, index = None)

In [78]:
# humanizing the search for each word in the keyword list


for word in checkedKeywordsList:
    print('Searching word: ',word)
    query = partA + word + partB
    response = requests.get(query)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #      to get the bugId, summary
    revRes = soup.findAll('a',{'class':['bz_bug_link', 'bz_status_RESOLVED','bz_closed']})

    #     to get the product and the component column results
    productRes = soup.findAll('td',{'class':['bz_product_column']})
    componentRes = soup.findAll('td',{'class':['bz_component_column']})
    
    #     keeping a track of the number of searches for each word in the keyword
    wordWiseResult = wordWiseResult.append({'word': word, 'Results found': len(revRes)},ignore_index=True)
    
    # we are assuming that the length of all the columns - bugId, product and component is the same.
    #     
    for revResValue in range(len(revRes)):        
    #     appending bugId, summary, keyword, product and component for each keyword
        taskDf = taskDf.append({'bugId':revRes[revResValue].text, 'summary':revRes[revResValue]['title'], 'keyword': word, 'product': productRes[revResValue].text.strip() , 'component': componentRes[revResValue].text.strip()},ignore_index=True)    
taskDf        

Searching word:  xss
Searching word:  overflow
Searching word:  oom
Searching word:  out%20of%20bound%20
Searching word:  valid
Searching word:  memori
Searching word:  memory
Searching word:  inject
Searching word:  use%20after%20fre%20
Searching word:  csrf
Searching word:  travers
Searching word:  pointer
Searching word:  authent
Searching word:  author
Searching word:  danger
Searching word:  permiss
Searching word:  critic
Searching word:  credenti
Searching word:  xml
Searching word:  xml%20external%20ent%20
Searching word:  xxe
Searching word:  deseri
Searching word:  privileg
Searching word:  secur
Searching word:  vulner
Searching word:  thread
Searching word:  crash


,summary,bugId,component,product,keyword
0,RESOLVED FIXED - XSS from booby-trapped link o...,1631739,General,Firefox for iOS,xss
1,RESOLVED FIXED - Javascript injection / XSS in...,1366420,Bookmarks & History,Firefox,xss
2,RESOLVED FIXED - Self-XSS message does not hav...,1565963,Console,DevTools,xss
3,RESOLVED FIXED - Improper parsing of QR codes ...,1567118,Browser,Firefox for iOS,xss
4,RESOLVED FIXED - [ ZDI-CAN-8375] UXSS priv-esc...,1538008,Sync,Firefox,xss
...,...,...,...,...,...
9252,RESOLVED FIXED - Crash in [@ mozilla::layers::...,1672621,Audio/Video: Playbac,Core,crash
9253,RESOLVED FIXED - GPUVideoTextureHost::EnsureWr...,1672858,Graphics: WebRender,Core,crash
9254,RESOLVED FIXED - Crash in [@ mozilla::gfx::Cro...,1682329,Graphics,Core,crash
9255,RESOLVED FIXED - Intermittent xpcshell <test-n...,1682884,XPCShell Harness,Testing,crash


In [79]:
# a = 'RESOLVED FIXED - CKM_CDMF_CBC out-of-bounds read with undersized IV'

# if 'out of bound' in a.replace('-',' ').lower() or 'out of bound' in a.replace('_',' ').lower():
#     print("yes")
#     print(a.replace('-',' ').lower())
#     print(a.replace('_',' ').lower())

In [80]:
keyword

['xss',
 'overflow',
 'oom',
 'out of bound',
 'valid',
 'memori',
 'memory',
 'inject',
 'use after fre',
 'csrf',
 'travers',
 'pointer',
 'authent',
 'author',
 'danger',
 'permiss',
 'critic',
 'credenti',
 'xml',
 'xml external ent',
 'xxe',
 'deseri',
 'privileg',
 'secur',
 'vulner',
 'thread',
 'crash']

In [81]:
# revResValue
import re
taskDf['presence'] = 'No'


for word in range(len(keyword)):
    for i in range(len(taskDf['summary'])):
#         if the word is 'oom' check for exact keyword in the string
        if keyword[word] == 'oom':
            if re.findall('\\boom\\b', taskDf['summary'][i].replace('-',' ').lower()) or re.findall('\\boom\\b', taskDf['summary'][i].replace('_',' ').lower()) or re.findall('\\booms\\b', taskDf['summary'][i].replace('-',' ').lower()) or re.findall('\\booms\\b', taskDf['summary'][i].replace('_',' ').lower()):
                taskDf['presence'][i] = 'Yes'
        else:
            if (keyword[word] in taskDf['summary'][i].replace('-',' ').lower()) or (keyword[word] in taskDf['summary'][i].replace('_',' ').lower()) :
                taskDf['presence'][i] = 'Yes'

In [93]:
# taskDf.reset_index()
newTaskDf = taskDf
taskDf.shape[0]

9257

In [101]:
for i in range((taskDf.shape[0])):
    if taskDf['presence'][i] == 'No':
        print(i)
        taskDf = taskDf.drop(i)

528
529
530
531
532
533
534
536
546
550
551
553
559
562
563
564
565
572
573
577
578
579
580
581
582
583
585
586
587
588
589
593
601
602
605
610
612
613
614
619
621
622
624
625
626
627
629
630
631
632
637
641
642
643
644
645
646
648
650
652
653
657
658
659
660
662
663
664
665
666
668
669
670
673
674
676
677
678
680
681
682
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
707
708
711
712
713
714
715
716
717
718
719
720
721
722
723
729
730
731
732
733
736
737
738
739
740
741
742
743
744
745
746
747
749
750
751
752
753
754
755
756
760
761
762
763
764
765
766
767
768
769
770
771
772
773
776
777
778
779
782
787
789
792
793
795
796
797
798
800
801
816
817
822
823
825
826
828
829
830
831
832
833
835
837
847
849
856
857
858
860
861
862
863
865
870
871
872
873
874
875
877
879
880
881
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
901
906
907
908
909
910
911
913
916
917
921
922
923
924
926
927
928
929
931
933
939
940
942
944
945
946
947
948


In [103]:
taskDf.reset_index()

,index,summary,bugId,component,product,keyword,presence
0,0,RESOLVED FIXED - XSS from booby-trapped link o...,1631739,General,Firefox for iOS,xss,Yes
1,1,RESOLVED FIXED - Javascript injection / XSS in...,1366420,Bookmarks & History,Firefox,xss,Yes
2,2,RESOLVED FIXED - Self-XSS message does not hav...,1565963,Console,DevTools,xss,Yes
3,3,RESOLVED FIXED - Improper parsing of QR codes ...,1567118,Browser,Firefox for iOS,xss,Yes
4,4,RESOLVED FIXED - [ ZDI-CAN-8375] UXSS priv-esc...,1538008,Sync,Firefox,xss,Yes
...,...,...,...,...,...,...,...
8854,9252,RESOLVED FIXED - Crash in [@ mozilla::layers::...,1672621,Audio/Video: Playbac,Core,crash,Yes
8855,9253,RESOLVED FIXED - GPUVideoTextureHost::EnsureWr...,1672858,Graphics: WebRender,Core,crash,Yes
8856,9254,RESOLVED FIXED - Crash in [@ mozilla::gfx::Cro...,1682329,Graphics,Core,crash,Yes
8857,9255,RESOLVED FIXED - Intermittent xpcshell <test-n...,1682884,XPCShell Harness,Testing,crash,Yes


In [111]:
taskDf.to_csv('task2-keywordSearch.csv')

In [109]:
# Unique Bug Id
print(len(taskDf['bugId'].unique()))


uniqueBugIdList = taskDf['bugId'].unique()

8134


# Patch extraction process (Work in progress)

In [107]:
# pd.read_excel('bug_report_data.xls')
brd = pd.read_csv('bug_report_data.csv')
bugIdList = []


for i in range(len(brd)):
    ff = brd['Bug Report'][i].split('?id=')
#     print(ff[1])
    bugIdList.append(ff[1])

In [108]:
print(len(bugIdList))
print(len(set(bugIdList)))

uniqueBugIdList = list(set(bugIdList))
len(uniqueBugIdList)

2738
2712


2712

In [ ]:
# # check the removing condition here

# for i in range(len(taskDf)):
#     if taskDf['bugId'][i] in uniqueBugIdList:
        
    

In [246]:
queryLink = 'https://bugzilla.mozilla.org/show_bug.cgi?id=1628291'
response = requests.get(queryLink)
soup = BeautifulSoup(response.text, 'html.parser')
revRes = soup.findAll('tr',{'class':['attach-patch']})
revRes

[<tr class="attach-patch">
 <td class="attach-desc-td">
 <div class="attach-desc">
 <a href="/attachment.cgi?id=9139851">Bug 1628291 - Catch exceptions when trying to create a Bitmap</a>
 </div>
 <div>
 <a class="attach-time activity-ref" href="#c2"><span class="rel-time" data-time="1586549558" title="2020-04-10 13:12 PDT">1 year ago</span></a>
 <span class="attach-author"><div class="vcard vcard_579431"><a class="email" href="/user_profile?user_id=579431"> <span class="fna">[:owlish] 🦉 PST</span></a>
 </div></span>
 </div>
 <div class="attach-info">47 bytes,
           text/x-phabricator-request        </div>
 </td>
 <td>
 </td>
 <td class="attach-actions">
 <a href="/attachment.cgi?id=9139851&amp;action=edit">Details</a>  |
   <a href="/attachment.cgi?id=9139851">Review</a>
 </td></tr>]

In [ ]:
# check for the patches here

for i in range(len(taskDf)):
    queryLink =  'https://bugzilla.mozilla.org/show_bug.cgi?id=' + taskDf['bugId'][i]    
    response = requests.get(queryLink)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #      to get the bugId, summary
    revRes = soup.findAll('a',{'class':['bz_bug_link', 'bz_status_RESOLVED','bz_closed']})

    

    

    

In [249]:
len(list(set(taskDf['bugId'])))

8890

In [243]:
c = 0
if '' in set(bugIdList):
    c = c+1
    
c

1

In [218]:
taskDf.to_csv('task2.csv')

In [7]:
wordWiseResult

,Results found,word
0,60,xss
1,463,overflow
2,601,oom
3,52,out%20of%20bound%20
4,865,valid
5,1,memori
6,503,memory
7,43,inject
8,341,use%20after%20fre%20
9,10,csrf


In [216]:
pwd

'/Users/chirag/Desktop/Research/Rolland Research/RollandNB'

In [114]:

query = 'https://bugzilla.mozilla.org/buglist.cgi?bug_status=RESOLVED&bug_status=VERIFIED&bug_status=CLOSED&chfieldfrom=2019-10-22&chfieldto=2021-01-26&classification=Client%20Software&classification=Developer%20Infrastructure&classification=Components&classification=Server%20Software&classification=Other&list_id=15772465&query_format=advanced&resolution=FIXED&short_desc=crash&short_desc_type=allwordssubstr&order=bug_status%2Cpriority%2Cassigned_to%2Cbug_id&limit=0'

# print(query)
response = requests.get(query)
soup = BeautifulSoup(response.text, 'html.parser')

productRes = soup.findAll('td',{'class':['bz_product_column']})
componentRes = soup.findAll('td',{'class':['bz_component_column']})

len(productRes) == len(componentRes)

for p in range(len(productRes)):
    print(productRes[p].text , " ------------ " , componentRes[p].text)